> ## European Soccer Database

### Data visualization

- **We going to use EUROPEAN FOOTBALL DATABASE for futhering data analysis**

---
**I) Cleaning and Preparing data**

---

**Discover all of the tables**

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

database = '../input/soccer/database.sqlite'
con = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", con)
tables

In [ ]:
countries = pd.read_sql_query("SELECT * from Country", con)
matches = pd.read_sql_query("SELECT * from Match", con)
leagues = pd.read_sql_query("SELECT * from League", con)
teams = pd.read_sql_query("SELECT * from Team", con)
player = pd.read_sql_query("SELECT * from Player",con)
player_attributes = pd.read_sql_query("SELECT * from Player_Attributes",con)
sequence = pd.read_sql_query("SELECT * from sqlite_sequence",con)
team_attributes = pd.read_sql_query("SELECT * from Team_Attributes",con)

**Import Data**

In [ ]:
countries.head()
leagues.head()
matches.head()
teams.head()
player.head()
player_attributes.head()
sequence.head()
team_attributes.head()

In [ ]:
countries_leagues = countries.merge(leagues,left_on="id",right_on="id",how="outer")
countries_leagues = countries_leagues.drop("id",axis = 1)
countries_leagues = countries_leagues.rename(columns={'name_x':"country", 'name_y':"league"})

In [ ]:
matches.head()

In [ ]:
#subsetting data with necessary columns
matches_new = matches[['id', 'country_id', 'league_id', 'season', 'stage', 'date',
                   'match_api_id', 'home_team_api_id', 'away_team_api_id',
                    'home_team_goal', 'away_team_goal']]

matches_new = matches_new.drop("id",axis=1)

In [ ]:
#merge leauge data with match data
data = matches_new.merge(countries_leagues,left_on="country_id",right_on="country_id",how="outer")
#Unique values in data
data.nunique()

---
#### Correlation of independent variables

In [ ]:
plt.figure(figsize=(17,10))
corr = data.corr()
ax = sns.heatmap(corr, annot=True, cmap='magma').set_title('Correlation of independent variables by using Pearson',fontsize=14)


---
#### Frequency score goals Home and away

In [ ]:
sns.set()
data.groupby("league").agg({"home_team_goal":"sum"
                            ,"away_team_goal":"sum"}).plot(kind="hist",figsize = (16,7),edgecolor = "k",linewidth =1)
plt.legend(loc = "best" , prop = {"size" : 14})
plt.xlabel("total goals")
plt.title('Frequency score goals Home and away',fontsize=14);

#### Matches by League

In [ ]:
sns.set()
plt.figure(figsize=(14,8))
ax = sns.countplot(y = data["league"],
                   order=data["league"].value_counts().index,
                   linewidth = 1,
                   edgecolor = "k"*data["league"].nunique()
                 )
for i,j in enumerate(data["league"].value_counts().values):
    ax.text(.8,i,j,weight = "bold")
plt.title("Matches by league")
plt.show()

---
#### MATCHES PLAYED IN EACH LEAGUE BY SEASON

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16.5,7))
ax1=data.groupby(['year']).agg({"home_team_api_id":["sum"]}).plot(ax=ax1, marker='.',linewidth=0.795,color='royalblue',markersize=17.23)
ax1.set_ylabel('Totally matches of home side ',fontsize=12)
ax2=data.groupby(['year']).agg({"away_team_api_id":["sum"]}).plot(ax=ax2, marker='.',linewidth=0.795,color='darkorange',markersize=17.23)
ax2.set_ylabel('Totally matches of away side',fontsize=12)
plt.show();

In [ ]:
plt.figure(figsize=(14,11))
sns.countplot(y = data["season"],hue=data["league"],palette="Dark2")
plt.title("MATCHES PLAYED IN EACH LEAGUE BY SEASON")
plt.show()